<a href="https://colab.research.google.com/github/supertime1/BERT/blob/main/QA_BERT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Import Dependencies

In [1]:
import tensorflow as tf